In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [23]:
import pandas as pd
path = "/content/drive/MyDrive/proyecto/"

ratings = pd.read_csv(path + "ratings.csv")

movies = pd.read_csv(path + "movies.csv")

print(movies.head())
print(ratings.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [24]:
df = pd.merge(ratings, movies, on='movieId')
df.drop(['timestamp'], axis=1, inplace=True)

df


,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...
100831,610,166534,4.0,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,Get Out (2017),Horror
100834,610,168252,5.0,Logan (2017),Action|Sci-Fi


In [ ]:
!pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 97.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.4 

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469543 sha256=9a7c0f9360ca05c168e9abf77fba3020ca5864e68a2fe624dbb57726b87af5f3
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


# Preparar datos y entrenar modelo KNN de Surprise:

In [4]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

lectura = Reader(rating_scale=(0.5, 5.0))
dataSet = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], lectura)

trainset, testset = train_test_split(dataSet, test_size=0.25)

sim_options = {'name': 'cosine', 'user_based': True}
modelo = KNNBasic(sim_options=sim_options)
modelo.fit(trainset)

predicciones = modelo.test(testset)
print("RMSE:", accuracy.rmse(predicciones))


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9721
RMSE: 0.9721364882470279


# Top 5 recomendaciones por usuario (ej. user 1)

In [5]:
from collections import defaultdict

def Top_recomenciones(predicciones, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predicciones:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = Top_recomenciones(predicciones, n=5)

user_1 = [iid for iid, _ in top_n[1]]
titulos_recomendados = movies[movies['movieId'].isin(user_1)]['title']
print("Películas recomendadas para el usuario 1:")
print(titulos_recomendados)


Películas recomendadas para el usuario 1:
900     Raiders of the Lost Ark (Indiana Jones and the...
911     Star Wars: Episode VI - Return of the Jedi (1983)
990             Indiana Jones and the Last Crusade (1989)
1939                                   Matrix, The (1999)
2226                                    Fight Club (1999)
Name: title, dtype: object


# Similaridad de películas por contenido (TF-IDF)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

movies['genres'] = movies['genres'].fillna('')
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()


# Función para recomendar películas similares

In [7]:
def recomendacion_pelicula(user_input, cosine_sim=cosine_sim):
    user_input = user_input.lower()
    matches = movies[movies['title'].str.lower().str.contains(user_input)]

    if matches.empty:
        return ["Película no encontrada"]

    idx = matches.index[0]
    title_found = movies.iloc[idx]['title']

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]

    recomendaciones = movies['title'].iloc[movie_indices].tolist()
    return title_found, recomendaciones


# Entrada del usuario y mostrar recomendaciones similares

In [8]:
entrada = input("¿Qué película te gusta? Escribe una parte del título: ")
resultado = recomendacion_pelicula(entrada)

if resultado[0] == "Película no encontrada":
    print("Película no encontrada.")
else:
    print(f"\nPelículas similares a '{resultado[0]}':")
    for r in resultado[1]:
        print("-", r)


¿Qué película te gusta? Escribe una parte del título: spider-man

Películas similares a 'Spider-Man (2002)':
- Independence Day (a.k.a. ID4) (1996)
- Escape from L.A. (1996)
- Abyss, The (1989)
- Escape from New York (1981)
- Star Trek: First Contact (1996)


# Preparar datos para la red neuronal (TensorFlow)

In [9]:
from sklearn.model_selection import train_test_split

# Copia para evitar modificar el DataFrame principal
df_nn = df.copy()

# Codificar userId y movieId como números consecutivos
df_nn['userId'] = df_nn['userId'].astype('category').cat.codes
df_nn['movieId'] = df_nn['movieId'].astype('category').cat.codes

X = df_nn[['userId', 'movieId']]
y = df_nn['rating']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Entrenar red neuronal artificial

In [10]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Crear y entrenar la red neuronal artificial
mlp = MLPRegressor(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    max_iter=500,
    random_state=42
)
mlp.fit(X_train, y_train)


MLPRegressor(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# Evaluar modelo con MAE y MSE

In [11]:
# Predecir sobre datos de prueba
y_pred = mlp.predict(X_test)

# Calcular métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")


MAE: 2.1044
MSE: 7.4770


# Predicción personalizada

In [12]:
user_id = X_test.iloc[0]['userId']
movie_id = X_test.iloc[0]['movieId']

# Hacer predicción
pred = mlp.predict([[user_id, movie_id]])
print(f"Predicción para userId={user_id}, movieId={movie_id}: {pred[0]:.2f}")

Predicción para userId=431, movieId=7316: 8.02


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


# Interacción con elección de tipo de recomendación

In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Opciones de tipo de recomendación
modo = widgets.RadioButtons(
    options=["Edad", "Categoría", "Película"],
    description='¿Filtrar por?',
    layout={'width': 'max-content'}
)

# Entradas
edad_input = widgets.BoundedIntText(value=18, min=3, max=100, description="Edad:")
categoria_input = widgets.Text(placeholder='Ej: terror, drama', description="Categoría:")
pelicula_input = widgets.Text(placeholder='Ej: Titanic', description="Película que te gusta:")

# Botón y salidas
btn_sugerir = widgets.Button(description="Ver sugerencias")
output = widgets.Output()
contenedor_inputs = widgets.VBox([])

# Cambiar entrada según selección
def on_change(change):
    if modo.value == "Edad":
        contenedor_inputs.children = [edad_input]
    elif modo.value == "Categoría":
        contenedor_inputs.children = [categoria_input]
    elif modo.value == "Película":
        contenedor_inputs.children = [pelicula_input]

modo.observe(on_change, names='value')

# Función para recomendar según edad
def filtrar_por_edad(edad):
    if edad <= 12:
        return ['Animation', 'Adventure', 'Family']
    elif edad <= 17:
        return ['Action', 'Comedy', 'Fantasy']
    elif edad <= 25:
        return ['Drama', 'Romance', 'Sci-Fi', 'Thriller']
    elif edad <= 40:
        return ['Drama', 'Mystery', 'Action', 'Comedy']
    else:
        return ['Documentary', 'History', 'Drama']

# Sugerencias principales
def sugerir_peliculas(b):
    with output:
        output.clear_output()
        generos = []

        if modo.value == "Edad":
            edad = edad_input.value
            generos = filtrar_por_edad(edad)
            print(f"🎯 Recomendación según edad {edad}: {generos}")

        elif modo.value == "Categoría":
            entrada = categoria_input.value.strip().title()
            if entrada:
                generos = [entrada]
                print(f"🎯 Recomendación según categoría: {generos}")

        elif modo.value == "Película":
            entrada = pelicula_input.value.strip().lower()
            coincidencias = movies[movies['title'].str.lower().str.contains(entrada)]
            if coincidencias.empty:
                print("⚠️ No se encontró la película.")
                return
            genero_base = coincidencias.iloc[0]['genres'].split('|')[0]  # primer género
            generos = [genero_base]
            print(f"🎯 Recomendación basada en '{coincidencias.iloc[0]['title']}' (género: {genero_base})")

        if not generos:
            print("⚠️ No se ingresó ningún criterio válido.")
            return

        sugeridas = movies[movies['genres'].str.contains('|'.join(generos), case=False)]
        sugeridas = sugeridas.sample(n=min(10, len(sugeridas)), random_state=1)

        if sugeridas.empty:
            print("⚠️ No se encontraron películas para esos géneros.")
        else:
            for titulo in sugeridas['title']:
                print("🎬", titulo)

# Conexión del botón
btn_sugerir.on_click(sugerir_peliculas)

# Mostrar interfaz
display(modo, contenedor_inputs, btn_sugerir, output)
on_change(None)  # mostrar input por defecto


RadioButtons(description='¿Filtrar por?', layout=Layout(width='max-content'), options=('Edad', 'Categoría', 'P…

VBox()

Button(description='Ver sugerencias', style=ButtonStyle())

Output()

In [30]:
import gradio as gr
import pandas as pd

df_movies =pd.read_csv(path + "movies.csv")

generos_por_edad = {
    10: ['Animation', 'Adventure', 'Family'],
    15: ['Comedy', 'Fantasy'],
    18: ['Drama', 'Romance', 'Thriller'],
    25: ['Action', 'Sci-Fi', 'Thriller'],
    40: ['Documentary', 'Drama', 'History']
}

def recomendar(tipo, edad, gustos, pelicula):
    if tipo == "Edad":
        edad_aprox = min(generos_por_edad.keys(), key=lambda x: abs(x - edad))
        generos = generos_por_edad[edad_aprox]
        rec = df_movies[df_movies['genres'].str.contains('|'.join(generos), case=False, na=False)]
        return f"🎂 Recomendaciones según edad {edad}:\n\n" + "\n".join(rec['title'].head(10).tolist())

    elif tipo == "Categoría":
        if not gustos:
            return "⚠️ Escribe al menos un género."
        gen = gustos.split(",")
        rec = df_movies
        for g in gen:
            rec = rec[rec['genres'].str.contains(g.strip(), case=False, na=False)]
        return f"🎬 Recomendaciones por género:\n\n" + "\n".join(rec['title'].head(10).tolist())

    elif tipo == "Película":
        if not pelicula:
            return "⚠️ Ingresa una película."
        pelicula = pelicula.lower()
        coincidencias = df_movies[df_movies['title'].str.lower().str.contains(pelicula)]
        if coincidencias.empty:
            return "❌ No se encontró esa película."
        genero = coincidencias.iloc[0]['genres'].split("|")[0]
        rec = df_movies[df_movies['genres'].str.contains(genero, case=False, na=False)]
        return f"🍿 Películas similares a '{coincidencias.iloc[0]['title']}' por género {genero}:\n\n" + "\n".join(rec['title'].head(10).tolist())

    return "❌ Opción no válida."

def actualizar_inputs(tipo):
    if tipo == "Edad":
        return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
    elif tipo == "Categoría":
        return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
    elif tipo == "Película":
        return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)
    else:
        return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

# ✅ Bloque completo Gradio con dinámica
with gr.Blocks() as demo:
    gr.Markdown("## 🎥 Recomendador Inteligente de Películas")
    tipo = gr.Radio(["Edad", "Categoría", "Película"], label="¿Cómo deseas la recomendación?", value="Edad")
    edad = gr.Number(label="Edad (solo si eliges 'Edad')", visible=True)
    gustos = gr.Textbox(label="Géneros favoritos (solo si eliges 'Categoría')", visible=False)
    peli = gr.Textbox(label="Película que te gustó (solo si eliges 'Película')", visible=False)

    btn = gr.Button("Ver sugerencias")
    salida = gr.Textbox(label="Películas recomendadas")

    # Conectar cambio de opción a visibilidad
    tipo.change(fn=actualizar_inputs, inputs=tipo, outputs=[edad, gustos, peli])
    btn.click(fn=recomendar, inputs=[tipo, edad, gustos, peli], outputs=salida)

demo.launch()
demo.launch(share=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://106357d30cfbab6f83.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://106357d30cfbab6f83.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pd.read_csv(path + "movies.csv")